
## Compute the associated average outward flux

To have a good quality gradient of the distance map, perform a light smooth over it. Define convolution kernels in each direction and use them recursively. 

In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from vmtk import vmtkscripts
from vmtk import vtkvmtk
import os
import sys
import numpy as np
from vtk.numpy_interface import dataset_adapter as dsa
import random
import networkx as nx
import vtk

from vtkdatawidgets.vtk_binding import VtkJupyterBridge

bridge = VtkJupyterBridge()
sphere = vtk.vtkSphereSource()
bridge.SetInputConnection(sphere.GetOutputPort())
bridge.Update()

imgr = vmtkscripts.vmtkImageReader()
imgr.InputFileName = os.path.join(os.getcwd(), 'test-surf-normal-hole.mha')
imgr.Execute()

mc = vmtkscripts.vmtkMarchingCubes()
mc.Image = imgr.Image
mc.Level = 1000.0
mc.Execute()

In [2]:
import math

In [3]:
surfr = vmtkscripts.vmtkSurfaceReader()
surfr.InputFileName = os.path.join(os.getcwd(), 'realtest.stl')
surfr.Execute()

surf = surfr.Surface

Reading STL surface file.


In [4]:
fedges = vtk.vtkFeatureEdges()
fedges.BoundaryEdgesOn()
fedges.FeatureEdgesOff()
fedges.ManifoldEdgesOff()
fedges.SetInputData(surf)
fedges.Update()

ofedges = fedges.GetOutput()

numEdges = ofedges.GetNumberOfPoints()

In [5]:
if numEdges != 0:
    tempcapper = vmtkscripts.vmtkSurfaceCapper()
    tempcapper.Surface = surf
    tempcapper.Interactive = 0
    tempcapper.Execute()
    
    networkSurface = tempcapper.Surface
else:
    networkSurface = surf

In [6]:
numCells = surf.GetNumberOfCells()
cellToDelete = random.randrange(0, numCells-1)

networkSurface.BuildLinks()
networkSurface.DeleteCell(cellToDelete)
networkSurface.RemoveDeletedCells()

net = vmtkscripts.vmtkNetworkExtraction()
net.Surface = networkSurface
net.AdvancementRatio = 1.001
net.Execute()

network = net.Network

Progress: 100%Progress: 100%


In [7]:
convert = vmtkscripts.vmtkCenterlinesToNumpy()
convert.Centerlines = network
convert.Execute()
ad = convert.ArrayDict

wrapping vtkPolyData object
converting cell data: 
Topology
converting points
converting point data: 
Radius
converting cell connectivity list


In [8]:
cellDataTopology = ad['CellData']['Topology']

In [9]:
cellDataTopology.shape

(102, 2)

In [10]:
fullTopology = np.array(cellDataTopology)

In [11]:
fullTopology.shape

(102, 2)

In [13]:
topologyToKeep = np.where(fullTopology[:,0] != 0)[0]

In [14]:
topol = fullTopology[topologyToKeep]

In [15]:
topol.shape

(101, 2)

In [16]:
topol

array([[ 1,  2],
       [ 2, -1],
       [ 2,  3],
       [ 3,  4],
       [ 3,  5],
       [ 4, -1],
       [ 5,  6],
       [ 5, -1],
       [ 6,  8],
       [ 6, -1],
       [ 7,  9],
       [ 7, 52],
       [ 8, 10],
       [ 9, 11],
       [ 9, 12],
       [10, 14],
       [10, 15],
       [11, 16],
       [11, -1],
       [12, 17],
       [12, 18],
       [13, 19],
       [13, 20],
       [14, -1],
       [14, -1],
       [15, 21],
       [16, 22],
       [16, -1],
       [17, 23],
       [17, 24],
       [18, -1],
       [18, -1],
       [19, -1],
       [19, -1],
       [20, -1],
       [21, 25],
       [21, -1],
       [22, -1],
       [22, -1],
       [23, 26],
       [24, 27],
       [24, -1],
       [25, 29],
       [25, 28],
       [26, 31],
       [26, -1],
       [27, 32],
       [28, 33],
       [29, -1],
       [30, 34],
       [30, 35],
       [31, 36],
       [31, 37],
       [32, 38],
       [32, -1],
       [34, 39],
       [34, -1],
       [35, -1],
       [35, 40

In [20]:
nodes = np.arange(1, topol.shape[0]+1)

In [21]:
nodes

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97])

In [22]:
topolWithNodes = np.copy(topol)

In [23]:
endEdgesToReplace = np.where(topolWithNodes[:, :] == -1)[0]

In [24]:
endEdgesToReplace

array([ 7, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 29, 33, 36, 38, 39, 40,
       43, 45, 47, 49, 51, 52, 54, 55, 56, 57, 60, 63, 68, 70, 72, 75, 78,
       79, 81, 85, 87, 88, 89, 90, 91, 92, 93, 94, 96])

In [26]:
for idx in endEdgesToReplace:
    if topolWithNodes[idx, 1] == -1:
        topolWithNodes[idx, 1] = nodes[idx]
    elif topolWithNodes[idx, 0] == -1:
        topolWithNodes[idx, 0] = nodes[idx]
    else:
        print('bad')

In [27]:
G = nx.Graph()

In [28]:
G.add_edges_from(topolWithNodes)

In [29]:
G.number_of_edges()

97

In [30]:
pos = nx.nx_agraph.graphviz_layout(G)
nx.draw(G, pos=pos, with_labels=True)

/Users/rick/miniconda3/envs/vmtkplayground/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [ ]:
for nodeidx in G:
    connections = [i for i in nx.all_neighbors(G, nodeidx)]
    if len(connections) < 2:
        print(f'{nodeidx}: {connections}')

In [ ]:
for node in G.nodes():
    print(f'{node}, {nx.}')

In [ ]:
n

In [ ]:
[i for i in C]

In [ ]:
pos = nx.nx_agraph.graphviz_layout(C)
nx.draw(C, pos=pos, with_labels=True)

In [ ]:
endCellIds = []
startCellIds = []
for cellId, cellTop in enumerate(cellDataTopology):
    if cellTop[1] == -1:
        endCellIds.append(cellId)

In [ ]:
endCellIds

In [ ]:
endPointIds = []
for cellId in endCellIds:  
    endPointIds.append(ad['CellData']['CellPointIds'][cellId][-2])

endPointCoordinates = []
for pointId in endPointIds:
    endPointCoordinates.append(ad['Points'][pointId])

In [ ]:
endPointCoordinates

In [ ]:
endPointCoordinates = np.array(endPointCoordinates)

In [ ]:
endPointCoordinates.shape

In [ ]:
sourcePoints = endPointCoordinates[-1]

In [ ]:
sourcePoints.shape

In [ ]:
flatSourcePoints = sourcePoints.flatten(order='C').tolist()

In [ ]:
sourcePoints

In [ ]:
targetPoints = endPointCoordinates[0:-1]

In [ ]:
targetPoints.shape

In [ ]:
flatTargetPoints = targetPoints.flatten(order='C').tolist()

In [ ]:
targetPoints

In [ ]:
cl = vmtkscripts.vmtkCenterlines()
cl.Surface = surf
cl.SeedSelectorName = 'pointlist'
cl.SourcePoints = flatSourcePoints
cl.TargetPoints = flatTargetPoints
cl.Execute()

numberOfCells = surfr.Surface.GetNumberOfCells()
cellToRemove = numberOfCells - 10
cellIds = vtk.vtkIdTypeArray()
cellIds.SetNumberOfComponents(1)
cellIds.InsertNextValue(cellToRemove)

selectionNode = vtk.vtkSelectionNode()
selectionNode.SetFieldType(selectionNode.CELL)
selectionNode.SetContentType(selectionNode.INDICES)
selectionNode.SetSelectionList(cellIds)

selection = vtk.vtkSelection()
selection.AddNode(selectionNode)

extractSelection = vtk.vtkExtractSelection()
extractSelection.SetInputData(0, surf)
extractSelection.SetInputData(1, selection)
extractSelection.Update()
    

select = vmtkscripts.vmtkSurfaceConnectivitySelector()
select.Surface = surfr.Surface
select.Execute()

In [ ]:
sv = vmtkscripts.vmtkSurfaceViewer()
sv.Surface = surf
sv.Execute()

In [ ]:
edit = vmtkscripts.vmtkNetworkEditor()
edit.Network = net.Network
edit.Execute()

In [ ]:
cv = vmtkscripts.vmtkCenterlineViewer()
cv.Centerlines = network
cv.Execute()

In [ ]:
conver

In [ ]:
rend = vmtkscripts.vmtkRenderer()
rend.Execute()

view = vmtkscripts.vmtkSurfaceViewer()
view.vmtkRenderer = rend.vmtkRenderer
view.Opacity = 0.3
view.Surface = surf
view.Execute()

view2 = vmtkscripts.vmtkCenterlineViewer()
view2.vmtkRenderer = rend.vmtkRenderer
view2.Centerlines = cl.Centerlines
view2.Execute()

In [ ]:
write = vmtkscripts.vmtkSurfaceWriter()
write.Surface = net.Network
write.OutputFileName = os.path.join(os.getcwd(), 'test-multi-hole.vtp')
write.Execute()